In [1]:
import warnings
warnings.filterwarnings(action="ignore")

import os
import numpy as np
import pandas as pd
from pyts.image import RecurrencePlot, GramianAngularField
from PIL import Image 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
def chunkify(df: pd.DataFrame, chunk_size: int, stride: int = 1):
    start = 0
    length = df.shape[0]

    # If DF is smaller than the chunk, return the DF
    if length <= chunk_size:
        return df[:]

    # Producing individual chunks
    dfs = []
    # while start + chunk_size <= length:
    #     dfs.append(df[start:chunk_size + start])
    #     start = start + chunk_size
    for i in range(0, length - chunk_size, stride):
        dfs.append(df[i:i + chunk_size])
    return dfs

In [3]:
df_rp_param_results = pd.read_csv('../data/results_rp_params_v5.csv')

In [4]:
# Seleção das linhas com o menor valor de c_ent_div_psvp para cada predictor
df_results = df_rp_param_results.loc[df_rp_param_results.groupby(['predictor','image'])['c_psvp'].idxmax()]

In [5]:
df_results.head()

,predictor,windows_size,optimal_delay,optimal_embedding,image,rr,c_ent_div_psvp,c_ent_mul_psvp,c_1_div_2_to_ent_div_psvp,c_1_div_2_to_ent_mul_psvp,c_ent,c_psvp
34,mains,448,1,1,gadf,0.00,4.459199,1.549779,6.605744e+06,0.001110,2.581463,0.203632
26,mains,224,1,1,gasf,0.00,10.095456,0.789784,1.482412e+02,0.004039,2.305287,0.114979
14,mains,112,1,1,rp,0.05,1.214083,0.706843,1.488209e+02,0.044875,0.910332,0.264288


In [6]:
# Calcular o MMC dos valores na coluna windows_size
window_sizes = df_results['windows_size'].values

In [7]:
df_value_value = pd.read_csv('../data/redd_active_value_f1hz.csv')
attributes = [c for c in df_value_value.columns.values if not c in ['timestamp']]
labels = [c for c in df_value_value.columns.values if not c in ['timestamp', 'mains', 'amplitude_spectrum', 'phase_spectrum']]
#labels = ['Fridge01','Dish washer01','Microwave01','Washer dryer01','Washer dryer02']
predictors = ['mains']
index_name = 'timestamp'
training_start = '2011-04-16'
training_end = '2011-05-16'
test_start = '2011-05-17'
test_end = '2011-05-31'

# Ensure 'timestamp' column is in datetime format
df_value_value[index_name] = pd.to_datetime(df_value_value[index_name])
# Set the index as the timestamp
df_value_value.set_index(index_name, inplace=True)

In [11]:
windows_size_mmc = np.lcm.reduce(window_sizes)
windows_size_opt = 112

In [24]:
#df_results_mmc = df_rp_param_results.loc[df_rp_param_results[df_rp_param_results['windows_size']==windows_size_mmc].groupby('predictor')['c_ent_div_psvp'].idxmin()]

In [12]:
df_results_opt = df_rp_param_results.loc[df_rp_param_results[df_rp_param_results['windows_size']==windows_size_opt].groupby('predictor')['c_psvp'].idxmax()]

In [26]:
#df_results_mmc

In [13]:
df_results_opt

,predictor,windows_size,optimal_delay,optimal_embedding,image,rr,c_ent_div_psvp,c_ent_mul_psvp,c_1_div_2_to_ent_div_psvp,c_1_div_2_to_ent_mul_psvp,c_ent,c_psvp
14,mains,112,1,1,rp,0.05,1.214083,0.706843,148.820947,0.044875,0.910332,0.264288


In [14]:
#data_train = {}
#for a in labels: # type: ignore
#    p = predictors.copy()
#    p.append(a)
#    data_train[a] = chunkify(
#            training_active_value_set[p], # type: ignore
#            windows_size_mmc,
#            windows_size_mmc
#        )
#test_active_value_set = df_binary_value.loc[test_start:test_end]
#test_active_value_set[predictors] = scaler.transform(test_active_value_set[predictors])
#data_test = {}
#for a in labels: # type: ignore
#    p = predictors.copy()
#    p.append(a)
#    data_test[a] = chunkify(
#            test_active_value_set[p], # type: ignore
#            windows_size_mmc,
#            windows_size_mmc
#        )

In [14]:
#image_path = '../data/v3'
#for p in predictors:
#    rp_image_transformer = RecurrencePlot(
#        dimension=df_results_mmc[df_results_mmc['predictor']==p]['optimal_embedding'].iloc[0],
#        time_delay=df_results_mmc[df_results_mmc['predictor']==p]['optimal_delay'].iloc[0],
#        percentage=int(df_results_mmc[df_results_mmc['predictor']==p]['rr'].iloc[0] * 10)
#    )
#    rgb_max = 0
#    rgb_min = 999
#    rgb_colors = 255 
#    for ds in data_train[labels[0]]: # type: ignore
#        result = rp_image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
#        rgb_max_aux = np.max(result[0])
#        rgb_min_aux = np.min(result[0])
#        if rgb_max < rgb_max_aux:
#                rgb_max = rgb_max_aux
#        if rgb_min > rgb_min_aux:
#                rgb_min = rgb_min_aux
#
#    i = 0
#    for ds in data_train[labels[0]]: # type: ignore
#        result = rp_image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
#        rgb_max_aux = np.max(result[0])
#        rgb_min_aux = np.min(result[0])
#
#        if (rgb_max_aux - rgb_min_aux) == 0:
#            matrix_rgb = np.full(result[0].shape, 0, dtype=np.uint8)
#        else:
#            matrix_normalized = (result[0] - rgb_min_aux) / (rgb_max_aux - rgb_min_aux)  # Normaliza a [0, 1]
#            matrix_rgb = (matrix_normalized * 255).astype(np.uint8)  # Escala a [0, 255] y convierte a uint8
#
#        pil_image = Image.fromarray(matrix_rgb)
#
#        # Redimensionar la imagen
#        pil_image = pil_image.resize((224, 224), Image.LANCZOS)
#
#        if not os.path.isdir(os.path.join(image_path)):
#            os.makedirs(os.path.join(image_path))      
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}'))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train'))
#        
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p)):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, str(0))):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, str(0)))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, str(1))):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, str(1)))
#        
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', 'all')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', 'all'))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, f'{i}.png')):
#            pil_image.save(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', p, f'{i}.png'))
#        
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', 'all',f'{i}_{p}.png')):
#            pil_image.save(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'train', 'all',f'{i}_{p}.png'))
#
#        i += 1
#
#    for ds in data_test[labels[0]]: # type: ignore
#        result = rp_image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
#        rgb_max_aux = np.max(result[0])
#        rgb_min_aux = np.min(result[0])
#        if rgb_max < rgb_max_aux:
#                rgb_max = rgb_max_aux
#        if rgb_min > rgb_min_aux:
#                rgb_min = rgb_min_aux
#
#    i = 0
#    for ds in data_test[labels[0]]: # type: ignore
#        result = rp_image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
#        rgb_max_aux = np.max(result[0])
#        rgb_min_aux = np.min(result[0])
#
#        if (rgb_max_aux - rgb_min_aux) == 0:
#            matrix_rgb = np.full(result[0].shape, 0, dtype=np.uint8)
#        else:
#            matrix_normalized = (result[0] - rgb_min_aux) / (rgb_max_aux - rgb_min_aux)  # Normaliza a [0, 1]
#            matrix_rgb = (matrix_normalized * 255).astype(np.uint8)  # Escala a [0, 255] y convierte a uint8
#
#        pil_image = Image.fromarray(matrix_rgb)
#
#        # Redimensionar la imagen
#        pil_image = pil_image.resize((224, 224), Image.LANCZOS)
#
#        if not os.path.isdir(os.path.join(image_path)):
#            os.makedirs(os.path.join(image_path))      
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}'))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test'))
#        
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', p)):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', p))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', 'all')):
#            os.makedirs(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', 'all'))
#
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', p, f'{i}.png')):
#            pil_image.save(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', p, f'{i}.png'))
#        
#        if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', 'all',f'{i}_{p}.png')):
#            pil_image.save(os.path.join(image_path, f'window_size_{windows_size_mmc}', 'test', 'all',f'{i}_{p}.png'))
#
#        i += 1
       


In [15]:
scaler = StandardScaler()
# Partition the DataFrame into training and test sets
training_active_value_set = df_value_value.loc[training_start:training_end]
training_active_value_set[predictors] = scaler.fit_transform(training_active_value_set[predictors])
data_train = {}
for a in labels: # type: ignore
    p = predictors.copy()
    p.append(a)
    data_train[a] = chunkify(
            training_active_value_set[p], # type: ignore
            windows_size_opt,
            10
        )
test_active_value_set = df_value_value.loc[test_start:test_end]
test_active_value_set[predictors] = scaler.transform(test_active_value_set[predictors])
data_test = {}
for a in labels: # type: ignore
    p = predictors.copy()
    p.append(a)
    data_test[a] = chunkify(
            test_active_value_set[p], # type: ignore
            windows_size_opt,
            10
        )

In [16]:
image_path = '../data/v5'
for p in predictors:
    
    for image_type in ['rp', 'gasf', 'gadf']:
    
        if image_type == 'rp':
            image_transformer = RecurrencePlot(
                dimension=int(df_results_opt[df_results_opt['predictor']==p]['optimal_embedding'].iloc[0]),
                time_delay=int(df_results_opt[df_results_opt['predictor']==p]['optimal_delay'].iloc[0]),
                percentage=int(df_results_opt[df_results_opt['predictor']==p]['rr'].iloc[0] * 100)
            )
        elif image_type == 'gasf':
            image_transformer = GramianAngularField(image_size=np.array(ds[p]).shape[0], method='summation')
        else:
            image_transformer = GramianAngularField(image_size=np.array(ds[p]).shape[0], method='difference')
        
        i = 0
        for ds in data_train[labels[0]]: # type: ignore
            result = image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
            
            matrix_rgb = (result[0] * 255).astype(np.uint8)  # Escala a [0, 255] y convierte a uint8

            pil_image = Image.fromarray(matrix_rgb)

            # Redimensionar la imagen
            pil_image = pil_image.resize((224, 224), Image.LANCZOS)

            if not os.path.isdir(os.path.join(image_path)):
                os.makedirs(os.path.join(image_path))      

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}'))

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train'))
            
            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', image_type)):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', image_type))

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', 'all')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', 'all'))

            if not os.path.isfile(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', 'all',f'{i}_{image_type}.png')):
                pil_image.save(os.path.join(image_path, f'window_size_{windows_size_opt}', 'train', 'all',f'{i}_{image_type}.png'))

            i += 1

        i = 0
        for ds in data_test[labels[0]]: # type: ignore
            result = image_transformer.fit_transform(np.array(ds[p]).reshape(1, -1)) # type: ignore
            
            matrix_rgb = (result[0] * 255).astype(np.uint8)  # Escala a [0, 255] y convierte a uint8

            pil_image = Image.fromarray(matrix_rgb)

            # Redimensionar la imagen
            pil_image = pil_image.resize((224, 224), Image.LANCZOS)

            if not os.path.isdir(os.path.join(image_path)):
                os.makedirs(os.path.join(image_path))      

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}'))

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test'))
            
            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', image_type)):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', image_type))

            if not os.path.isdir(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', 'all')):
                os.makedirs(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', 'all'))

            if not os.path.isfile(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', 'all',f'{i}_{image_type}.png')):
                pil_image.save(os.path.join(image_path, f'window_size_{windows_size_opt}', 'test', 'all',f'{i}_{image_type}.png'))

            i += 1
       
